In [ ]:
#Description: Predicting Ethereum Price with Python and Machine Learning

In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# Import required libraries
from datetime import datetime
import requests

In [ ]:
# Define the API endpoint and parameters
endpoint = '[API Link]'
params = {
    'vs_currency': 'usd',
    'days': 'max'
}
